##Project: Dive Deeper into Web Scraping

© 2024, Zaka AI, Inc. All Rights Reserved.

---

###**Case Study:** Netflix Top 50
**Objective:** The goal of this challenge is to let you go over a project that allows you to dive deeper into webscraping and learn some new related concepts.

* You will scrape the website https://www.digitaltrends.com/movies/best-movies-on-netflix/ using Beautifulsoup, and get the required information about the presented movies. Once you clean them, you would save them in a SQL database, and run some queries over them.


##Scraping the best movies on Netflix from the digitaltrends website

You will get the necessary information about the movies on the website an save them in a SQL database. Before doing that, you need to familiarize yourself with the sqlite3 library

####1. Familiarize yourself with sqlite3.

The goal of this section is just for you to practice the sqlite3 python's library.
Start by importing the library and creating a new database.

In [ ]:
#Test your Zaka
import sqlite3
conn = sqlite3.connect('my_database.db')

Create a table in this database. This table should contain a few columns. You can add whatever columns you want. The goal is just to practice.

In [ ]:
#Test your Zaka
cursor = conn.cursor()

# Create a new table
cursor.execute('''
CREATE TABLE IF NOT EXISTS employees (
    employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    department TEXT,
    salary REAL,
    hire_date TEXT
)
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Table 'employees' created successfully!")


Table 'employees' created successfully!


Add data to the table you created

In [ ]:
#Test your Zaka
# Connect to the database
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

# Insert data into the table
cursor.execute('''
INSERT INTO employees (first_name, last_name, department, salary, hire_date)
VALUES ('John', 'Doe', 'Engineering', 75000.00, '2022-10-15')
''')

cursor.execute('''
INSERT INTO employees (first_name, last_name, department, salary, hire_date)
VALUES ('Jane', 'Smith', 'Marketing', 65000.00, '2021-06-20')
''')

cursor.execute('''
INSERT INTO employees (first_name, last_name, department, salary, hire_date)
VALUES ('Alice', 'Brown', 'HR', 58000.00, '2023-03-01')
''')

# Commit changes and close the connection
conn.commit()
conn.close()

print("Data added to the 'employees' table successfully!")

Data added to the 'employees' table successfully!


Perform a few basic queries (2-3) to familiarize yourself with the usage (The queries can be anything like selecting all rows in the table or selecting a specific column according to a condition, etc.)

In [ ]:
#Test your Zaka
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()
cursor.execute('SELECT first_name, last_name FROM employees WHERE salary > 60000')
rows = cursor.fetchall()
print("Employees with salary greater than 60000:")
for row in rows:
  print(row)
conn.close()

Employees with salary greater than 60000:
('John', 'Doe')
('Jane', 'Smith')


In [ ]:
#Test your Zaka
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()
cursor.execute("SELECT first_name , last_name FROM employees WHERE salary > 50000 AND department = 'HR'")
rows = cursor.fetchall()
print("Employess with salary greater than 50000 and work as HR:")
for row in rows:
  print(row)
conn.close()

Employess with salary greater than 50000 and work as HR:
('Alice', 'Brown')


In [ ]:
#Test your Zaka
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()
cursor.execute('SELECT first_name, last_name FROM employees WHERE salary > 60000')
rows = cursor.fetchall()
print("Employees with salary greater than 60000:")
for row in rows:
  print(row)
conn.close()

Employees with salary greater than 60000:
('John', 'Doe')
('Jane', 'Smith')


###2. Scraping the website via BeautifulSoup.

The website to scrape is the following https://www.digitaltrends.com/movies/best-movies-on-netflix/. Start by sending a request to the website and make sure you obtain the right response.

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# URL of the page to scrape
url = "https://www.digitaltrends.com/movies/best-movies-on-netflix/"  # Replace with your target URL

# Send a GET request to fetch the page content
response = requests.get(url)



In [ ]:
import sqlite3

For each movie, extract the following:
* Title
* Year
* Poster
* Duration
* IMDB Rating
* Genre
* Stars (Actors)
* Director TEXT

**Tip:** You can start by extracting each element alone to make sure everything is function as expected and then wrap the whole thing within a function when you need to save them into a database

In [ ]:
#Test your Zaka
soup = BeautifulSoup(response.content, 'html.parser')

# Prettify the soup for better readability
soup.prettify()

# Extract Titles and Years
all_titles = soup.find_all("h3", class_="b-media__title")
movie_titles = []
release_years = []

for title_text in all_titles:
    title = title_text.text.strip()
    year_match = re.search(r'\(\d{4}\)', title)
    if year_match:
        year = year_match.group()
        title = title.replace(year, '').strip()
        year = year.replace('(', '').replace(')', '')

        movie_titles.append(title)
        release_years.append(year)

# Extract Poster URLs
all_posters = soup.find_all("div", class_="b-media__poster")
poster_urls = []

for poster_div in all_posters:
    img = poster_div.find('img')
    if img:
        img_url = img.get("data-dt-lazy-src")
        poster_urls.append(img_url)

# Extract Durations
all_durations = soup.find_all("div", class_="b-media__basic")
durations = []

for duration in all_durations:
    duration_text = duration.find_all(string=True)
    for text in duration_text:
        if "m" in text:  # Check if the duration contains 'm' (minutes)
            duration_clean = re.sub(r"[^0-9]", "", text.strip())  # Remove everything except digits
            durations.append(duration_clean)  # Append the cleaned duration
            break


# Extract IMDB Ratings
all_ratings = soup.find_all('div', class_='b-media__score')
movie_ratings = []

for rating_div in all_ratings:
    rating_text = rating_div.text.strip()
    if "/10" in rating_text:
        movie_ratings.append(rating_text)

# Extract Genres
all_genres = soup.find_all("div", class_="b-media__key")
movie_genres = []

for genre_div in all_genres:
    if genre_div.text.strip() == "Genre":
        value_div = genre_div.find_next_sibling("div", class_="b-media__value")
        if value_div:
            movie_genres.append(value_div.text.strip())

# Extract Actors (Stars)
all_actors = soup.find_all("div", class_="b-media__key")
movie_actors = []

for actor_div in all_actors:
    if actor_div.text.strip() == "Stars":
        value_div = actor_div.find_next_sibling("div", class_="b-media__value")
        if value_div:
            movie_actors.append(value_div.text.strip())

# Extract Directors
all_directors = soup.find_all("div", class_="b-media__key")
movie_directors = []

for director_div in all_directors:
    if director_div.text.strip() == "Directed by":
        value_div = director_div.find_next_sibling("div", class_="b-media__value")
        if value_div:
            movie_directors.append(value_div.text.strip())

# Combine all data into a list of dictionaries
movies_data = []

for i in range(len(movie_titles)):
    movie_data = {
        'Title': movie_titles[i],
        'Year': release_years[i],
        'Poster': poster_urls[i] if i < len(poster_urls) else None,
        'Duration': durations[i] if i < len(durations) else None, # Use 'durations' instead of 'movie_durations'
        'IMDB Rating': movie_ratings[i] if i < len(movie_ratings) else None,
        'Genre': movie_genres[i] if i < len(movie_genres) else None,
        'Stars': movie_actors[i] if i < len(movie_actors) else None,
        'Director': movie_directors[i] if i < len(movie_directors) else None
    }
    movies_data.append(movie_data)

# Output the extracted movie data (for testing purposes)
for movie in movies_data:
    print(movie)


{'Title': 'Scream VI', 'Year': '2023', 'Poster': 'https://www.digitaltrends.com/wp-content/uploads/2025/01/wDWwtvkRRlgTiUr6TyLSMX8FCuZ.jpg?p=1#038;p=1.jpg', 'Duration': '123', 'IMDB Rating': '6.4/10', 'Genre': 'Horror, Crime, Thriller', 'Stars': 'Melissa Barrera, Jenna Ortega, Jasmin Savoy Brown', 'Director': 'Tyler Gillett, Matt Bettinelli-Olpin'}
{'Title': 'Saturday Night', 'Year': '2024', 'Poster': 'https://www.digitaltrends.com/wp-content/uploads/2025/01/oCf5O6uxooTvRwKVnLHwGqZUifq.jpg?p=1#038;p=1.jpg', 'Duration': '109', 'IMDB Rating': '7.0/10', 'Genre': 'Comedy, Drama', 'Stars': 'Gabriel LaBelle, Rachel Sennott, Cory Michael Smith', 'Director': 'Jason Reitman'}
{'Title': 'Back in Action', 'Year': '2025', 'Poster': 'https://www.digitaltrends.com/wp-content/uploads/2025/01/zDdwuxEr6GTtl0o7PbVvVo4ht5i.jpg?p=1#038;p=1.jpg', 'Duration': '114', 'IMDB Rating': '7.3/10', 'Genre': 'Action, Comedy', 'Stars': 'Cameron Diaz, Jamie Foxx, McKenna Roberts', 'Director': 'Seth Gordon'}
{'Title': 

In [ ]:
len(movies_data)

50

####*Save the result in a SQL database*

In [ ]:
#Test your Zaka
import sqlite3

# Connect to (or create) an SQLite database file
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Create a table for storing the movie data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS movies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        year INTEGER,
        poster_url TEXT,
        duration INTEGER,
        imdb_rating TEXT,
        genre TEXT,
        stars TEXT,
        director TEXT
    )
''')

# Commit the changes
conn.commit()
# Insert movie data into the database
for movie in movies_data:
    cursor.execute('''
        INSERT INTO movies (title, year, poster_url, duration, imdb_rating, genre, stars, director)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        movie['Title'],
        movie['Year'],
        movie['Poster'],
        movie['Duration'],
        movie['IMDB Rating'],
        movie['Genre'],
        movie['Stars'],
        movie['Director']
    ))

# Commit the changes and close the connection
conn.commit()
conn.close()


In [ ]:
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM movies LIMIT 5')  # Retrieve the first 5 rows
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()


(1, 'Scream VI', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/wDWwtvkRRlgTiUr6TyLSMX8FCuZ.jpg?p=1#038;p=1.jpg', 123, '6.4/10', 'Horror, Crime, Thriller', 'Melissa Barrera, Jenna Ortega, Jasmin Savoy Brown', 'Tyler Gillett, Matt Bettinelli-Olpin')
(2, 'Saturday Night', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/oCf5O6uxooTvRwKVnLHwGqZUifq.jpg?p=1#038;p=1.jpg', 109, '7.0/10', 'Comedy, Drama', 'Gabriel LaBelle, Rachel Sennott, Cory Michael Smith', 'Jason Reitman')
(3, 'Back in Action', 2025, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/zDdwuxEr6GTtl0o7PbVvVo4ht5i.jpg?p=1#038;p=1.jpg', 114, '7.3/10', 'Action, Comedy', 'Cameron Diaz, Jamie Foxx, McKenna Roberts', 'Seth Gordon')
(4, 'Hereditary', 2018, 'https://www.digitaltrends.com/wp-content/uploads/2023/04/p9fmuz2Oj3HtEJEqbIwkFGUhVXD.jpg?p=1#038;p=1.jpg', 128, '7.5/10', 'Horror, Mystery, Thriller', 'Toni Collette, Alex Wolff, Gabriel Byrne', 'Ari Aster')
(5, 'Furiosa: A Mad Max Saga', 2

####3. Running some queries against the database we created

Write a query that returns all movies with all their specifications

In [ ]:
#Test your Zaka
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to select all movies
cursor.execute('SELECT * FROM movies')
rows = cursor.fetchall()

# Display all movies
for row in rows:
    print(row)

conn.close()


(1, 'Scream VI', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/wDWwtvkRRlgTiUr6TyLSMX8FCuZ.jpg?p=1#038;p=1.jpg', 123, '6.4/10', 'Horror, Crime, Thriller', 'Melissa Barrera, Jenna Ortega, Jasmin Savoy Brown', 'Tyler Gillett, Matt Bettinelli-Olpin')
(2, 'Saturday Night', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/oCf5O6uxooTvRwKVnLHwGqZUifq.jpg?p=1#038;p=1.jpg', 109, '7.0/10', 'Comedy, Drama', 'Gabriel LaBelle, Rachel Sennott, Cory Michael Smith', 'Jason Reitman')
(3, 'Back in Action', 2025, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/zDdwuxEr6GTtl0o7PbVvVo4ht5i.jpg?p=1#038;p=1.jpg', 114, '7.3/10', 'Action, Comedy', 'Cameron Diaz, Jamie Foxx, McKenna Roberts', 'Seth Gordon')
(4, 'Hereditary', 2018, 'https://www.digitaltrends.com/wp-content/uploads/2023/04/p9fmuz2Oj3HtEJEqbIwkFGUhVXD.jpg?p=1#038;p=1.jpg', 128, '7.5/10', 'Horror, Mystery, Thriller', 'Toni Collette, Alex Wolff, Gabriel Byrne', 'Ari Aster')
(5, 'Furiosa: A Mad Max Saga', 2

Write a query that selects movies having an IMDB rating higher than 8.5

In [ ]:
#Test your Zaka
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to select movies with an IMDB rating higher than 8.5
cursor.execute('''
    SELECT * FROM movies
    WHERE CAST(SUBSTR(imdb_rating, 1, INSTR(imdb_rating, '/') - 1) AS REAL) > 8.5
''')

rows = cursor.fetchall()

# Display the movies
for row in rows:
    print(row)

conn.close()


(6, 'In the Heart of the Sea', 2015, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/weVvNPfk6FM9vBg3BXtRtNAmiYM.jpg?p=1#038;p=1.jpg', 122, '8.7/10', 'Thriller, Drama, Adventure, Action, History', 'Chris Hemsworth, Benjamin Walker, Cillian Murphy', 'Ron Howard')
(45, 'Dumb Money', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/01/bqs4TeKFbfkcbQbuGAuKYu82tnp.jpg?p=1#038;p=1.jpg', 104, '8.7/10', 'Comedy, Drama, History', 'Paul Dano, Shailene Woodley, America Ferrera', 'Craig Gillespie')
(56, 'In the Heart of the Sea', 2015, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/weVvNPfk6FM9vBg3BXtRtNAmiYM.jpg?p=1#038;p=1.jpg', 122, '8.7/10', 'Thriller, Drama, Adventure, Action, History', 'Chris Hemsworth, Benjamin Walker, Cillian Murphy', 'Ron Howard')
(95, 'Dumb Money', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/01/bqs4TeKFbfkcbQbuGAuKYu82tnp.jpg?p=1#038;p=1.jpg', 104, '8.7/10', 'Comedy, Drama, History', 'Paul Dano, Shailene Woodley, America 

Write a query that selects only comedy movies

In [ ]:
#Test your Zaka
# Test your Zaka
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to select comedy movies and order them by IMDb rating in descending order
cursor.execute("SELECT * FROM movies WHERE genre LIKE '%Comedy%' ORDER BY imdb_rating DESC")
rows = cursor.fetchall()

# Display the comedy movies
for row in rows:
    print(row)

conn.close()



(45, 'Dumb Money', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/01/bqs4TeKFbfkcbQbuGAuKYu82tnp.jpg?p=1#038;p=1.jpg', 104, '8.7/10', 'Comedy, Drama, History', 'Paul Dano, Shailene Woodley, America Ferrera', 'Craig Gillespie')
(95, 'Dumb Money', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/01/bqs4TeKFbfkcbQbuGAuKYu82tnp.jpg?p=1#038;p=1.jpg', 104, '8.7/10', 'Comedy, Drama, History', 'Paul Dano, Shailene Woodley, America Ferrera', 'Craig Gillespie')
(35, 'Beverly Hills Cop: Axel F', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2024/07/zszRKfzjM5jltiq8rk6rasKVpUv.jpg?p=1#038;p=1.jpg', 118, '7.9/10', 'Action, Comedy, Crime', 'Eddie Murphy, Joseph Gordon-Levitt, Taylour Paige', 'Mark Molloy')
(85, 'Beverly Hills Cop: Axel F', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2024/07/zszRKfzjM5jltiq8rk6rasKVpUv.jpg?p=1#038;p=1.jpg', 118, '7.9/10', 'Action, Comedy, Crime', 'Eddie Murphy, Joseph Gordon-Levitt, Taylour Paige', 'Mark Molloy')
(16, '

Write a query that selects comedy or action movies shorter than 120 minutes.

In [ ]:
#Test your Zaka
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to select comedy or action movies shorter than 120 minutes
cursor.execute('''
    SELECT * FROM movies
    WHERE (genre LIKE '%Comedy%' OR genre LIKE '%Action%'
    AND duration < 120)
''')

rows = cursor.fetchall()

# Display the results
for row in rows:
    print(row)

conn.close()


(2, 'Saturday Night', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/oCf5O6uxooTvRwKVnLHwGqZUifq.jpg?p=1#038;p=1.jpg', 109, '7.0/10', 'Comedy, Drama', 'Gabriel LaBelle, Rachel Sennott, Cory Michael Smith', 'Jason Reitman')
(3, 'Back in Action', 2025, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/zDdwuxEr6GTtl0o7PbVvVo4ht5i.jpg?p=1#038;p=1.jpg', 114, '7.3/10', 'Action, Comedy', 'Cameron Diaz, Jamie Foxx, McKenna Roberts', 'Seth Gordon')
(8, 'Wallace & Gromit: Vengeance Most Fowl', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/6BxK38ehxuX2dJmZIMpJcVNbYks.jpg?p=1#038;p=1.jpg', 79, '6.1/10', 'Animation, Comedy, Family', 'Ben Whitehead, Reece Shearsmith, Peter Kay', 'Nick Park, Merlin Crossingham')
(9, 'Godzilla x Kong: The New Empire', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2024/07/z1p34vh7dEOnLDmyCrlUVLuoDzd.jpg?p=1#038;p=1.jpg', 115, '6.8/10', 'Action, Adventure, Science Fiction', 'Rebecca Hall, Brian Tyree Henry, Dan Steven

Add as much queries as you want to derive insights from your database.
Remember you will share these insights through your presentation.

In [ ]:
#Test your Zaka
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to check if Jason Statham is among the stars in the first 50 movies
cursor.execute('''
    SELECT * FROM movies
    WHERE stars LIKE '%Jason Statham%'
    LIMIT 50
''')

rows = cursor.fetchall()

# Display the results
if rows:
    for row in rows:
        print(row)
else:
    print("Jason Statham is not listed in the first 50 movies.")

conn.close()


Jason Statham is not listed in the first 50 movies.


In [ ]:
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to check if Margot Robbie is among the stars
cursor.execute('''
    SELECT * FROM movies
    WHERE stars LIKE '%Margot Robbie%'
''')

rows = cursor.fetchall()

# Display the results
if rows:
    for row in rows:
        print(row)
else:
    print("Margot Robbie is not listed in any movie.")

conn.close()


(10, 'Barbie', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2023/12/iuFNMS8U5cb6xfzi51Dbkovj7vM.jpg?p=1#038;p=1.jpg', 114, '5.6/10', 'Comedy, Adventure', 'Margot Robbie, Ryan Gosling, America Ferrera', 'Greta Gerwig')
(60, 'Barbie', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2023/12/iuFNMS8U5cb6xfzi51Dbkovj7vM.jpg?p=1#038;p=1.jpg', 114, '5.6/10', 'Comedy, Adventure', 'Margot Robbie, Ryan Gosling, America Ferrera', 'Greta Gerwig')


In [ ]:
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to find all Scream movies and sort by IMDb rating
cursor.execute('''
    SELECT * FROM movies
    WHERE title LIKE '%Scream%'
    ORDER BY imdb_rating DESC
''')

rows = cursor.fetchall()

# Display the results
if rows:
    print("All Scream movies, sorted by IMDb rating:")
    for row in rows:
        print(row)
else:
    print("No Scream movies found.")

conn.close()


All Scream movies, sorted by IMDb rating:
(26, 'Scream', 2022, 'https://www.digitaltrends.com/wp-content/uploads/2024/10/1m3W6cpgwuIyjtg5nSnPx7yFkXW.jpg?p=1#038;p=1.jpg', 114, '7.0/10', 'Horror, Mystery, Thriller', 'Melissa Barrera, Jenna Ortega, Mason Gooding', 'Tyler Gillett, Matt Bettinelli-Olpin')
(76, 'Scream', 2022, 'https://www.digitaltrends.com/wp-content/uploads/2024/10/1m3W6cpgwuIyjtg5nSnPx7yFkXW.jpg?p=1#038;p=1.jpg', 114, '7.0/10', 'Horror, Mystery, Thriller', 'Melissa Barrera, Jenna Ortega, Mason Gooding', 'Tyler Gillett, Matt Bettinelli-Olpin')
(1, 'Scream VI', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/wDWwtvkRRlgTiUr6TyLSMX8FCuZ.jpg?p=1#038;p=1.jpg', 123, '6.4/10', 'Horror, Crime, Thriller', 'Melissa Barrera, Jenna Ortega, Jasmin Savoy Brown', 'Tyler Gillett, Matt Bettinelli-Olpin')
(51, 'Scream VI', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/wDWwtvkRRlgTiUr6TyLSMX8FCuZ.jpg?p=1#038;p=1.jpg', 123, '6.4/10', 'Horror, Crime, Thrille

**bold text**
One of my favourite movies is the equalizer ,lets check if it is part of the best 50 movies

In [ ]:
import sqlite3

try:
    # Connect to the database
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    # Query to find all movies with "Equalizer" in the title
    cursor.execute('''
        SELECT * FROM movies
        WHERE title LIKE '%Equalizer%'
    ''')

    rows = cursor.fetchall()

    # Display results
    if rows:
        print("Movies with 'Equalizer' in the title:")
        for row in rows:
            print(row)
    else:
        print("No movies with 'Equalizer' found.")

except sqlite3.Error as e:
    print(f"Database error: {e}")

finally:
    # Close the database connection
    conn.close()


Movies with 'Equalizer' in the title:
(46, 'The Equalizer 3', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/01/b0Ej6fnXAP8fK75hlyi2jKqdhHz.jpg?p=1#038;p=1.jpg', 109, '6.4/10', 'Action, Thriller, Crime', 'Denzel Washington, Dakota Fanning, Eugenio Mastrandrea', 'Antoine Fuqua')
(96, 'The Equalizer 3', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/01/b0Ej6fnXAP8fK75hlyi2jKqdhHz.jpg?p=1#038;p=1.jpg', 109, '6.4/10', 'Action, Thriller, Crime', 'Denzel Washington, Dakota Fanning, Eugenio Mastrandrea', 'Antoine Fuqua')


Lets check the best 5 action and horror movies based on the ratings

In [ ]:
import sqlite3

try:
    # Connect to the database
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    # Query for best 5 action movies
    cursor.execute('''
        SELECT * FROM movies
        WHERE genre LIKE '%Action%'
        ORDER BY imdb_rating DESC
        LIMIT 5
    ''')
    best_action_movies = cursor.fetchall()

    # Query for best 5 horror movies
    cursor.execute('''
        SELECT * FROM movies
        WHERE genre LIKE '%Horror%'
        ORDER BY imdb_rating DESC
        LIMIT 5
    ''')
    best_horror_movies = cursor.fetchall()

    # Display results
    print("Best 5 Action Movies:")
    if best_action_movies:
        for movie in best_action_movies:
            print(movie)
    else:
        print("No action movies found.")

    print("\nBest 5 Horror Movies:")
    if best_horror_movies:
        for movie in best_horror_movies:
            print(movie)
    else:
        print("No horror movies found.")

except sqlite3.Error as e:
    print(f"Database error: {e}")

finally:
    # Close the database connection
    conn.close()


Best 5 Action Movies:
(6, 'In the Heart of the Sea', 2015, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/weVvNPfk6FM9vBg3BXtRtNAmiYM.jpg?p=1#038;p=1.jpg', 122, '8.7/10', 'Thriller, Drama, Adventure, Action, History', 'Chris Hemsworth, Benjamin Walker, Cillian Murphy', 'Ron Howard')
(56, 'In the Heart of the Sea', 2015, 'https://www.digitaltrends.com/wp-content/uploads/2025/01/weVvNPfk6FM9vBg3BXtRtNAmiYM.jpg?p=1#038;p=1.jpg', 122, '8.7/10', 'Thriller, Drama, Adventure, Action, History', 'Chris Hemsworth, Benjamin Walker, Cillian Murphy', 'Ron Howard')
(13, 'Carry-On', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2024/12/jXLtHkDo1tRBxnxALTPd9AiH2bH.jpg?p=1#038;p=1.jpg', 119, '8.1/10', 'Thriller, Mystery, Action', 'Taron Egerton, Jason Bateman, Sofia Carson', 'Jaume Collet-Serra')
(63, 'Carry-On', 2024, 'https://www.digitaltrends.com/wp-content/uploads/2024/12/jXLtHkDo1tRBxnxALTPd9AiH2bH.jpg?p=1#038;p=1.jpg', 119, '8.1/10', 'Thriller, Mystery, Action', 'Taron Egerto

**Lets check for movies that are considered to be " horror and action  ", or " horror and comedy" at the same time**

In [ ]:


try:
    # Connect to the database
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()

    # Query for movies that are both Horror and Action
    cursor.execute('''
        SELECT * FROM movies
        WHERE genre LIKE '%Horror%' AND genre LIKE '%Action%'
        ORDER BY imdb_rating DESC
    ''')
    horror_action_movies = cursor.fetchall()

    # Query for movies that are both Horror and Comedy
    cursor.execute('''
        SELECT * FROM movies
        WHERE genre LIKE '%Horror%' AND genre LIKE '%Comedy%'
        ORDER BY imdb_rating DESC
    ''')
    horror_comedy_movies = cursor.fetchall()

    # Display results
    print("Horror and Action Movies:")
    if horror_action_movies:
        for movie in horror_action_movies:
            print(movie)
    else:
        print("No Horror and Action movies found.")

    print("\nHorror and Comedy Movies:")
    if horror_comedy_movies:
        for movie in horror_comedy_movies:
            print(movie)
    else:
        print("No Horror and Comedy movies found.")

except sqlite3.Error as e:
    print(f"Database error: {e}")

finally:
    # Close the database connection
    conn.close()


Horror and Action Movies:
(37, 'Godzilla Minus One', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/06/hkxxMIGaiCTmrEArK7J56JTKUlB.jpg?p=1#038;p=1.jpg', 125, '7.5/10', 'Science Fiction, Horror, Action', 'Ryunosuke Kamiki, Minami Hamabe, Yuki Yamada', 'Takashi Yamazaki')
(87, 'Godzilla Minus One', 2023, 'https://www.digitaltrends.com/wp-content/uploads/2024/06/hkxxMIGaiCTmrEArK7J56JTKUlB.jpg?p=1#038;p=1.jpg', 125, '7.5/10', 'Science Fiction, Horror, Action', 'Ryunosuke Kamiki, Minami Hamabe, Yuki Yamada', 'Takashi Yamazaki')

Horror and Comedy Movies:
No Horror and Comedy movies found.


bold text Lets check if Vin Deisel participated among one of the best 50 movies


In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# Query to find top 50 movies by IMDb rating and check if Vin Diesel is among the stars
cursor.execute('''
    SELECT title, imdb_rating, stars
    FROM movies
    WHERE stars LIKE '%Vin Diesel%'
    ORDER BY imdb_rating DESC
    LIMIT 50
''')

# Fetch the results
movies = cursor.fetchall()

# Display the movies that have Vin Diesel and their IMDb rating
if movies:
    print("Movies with Vin Diesel in the top 50 based on IMDb rating:")
    for movie in movies:
        title = movie[0]
        imdb_rating = movie[1]
        stars = movie[2]
        print(f"Title: {title}, IMDb Rating: {imdb_rating}, Stars: {stars}")
else:
    print("No movies with Vin Diesel in the top 50 were found.")

# Close the connection
conn.close()


Movies with Vin Diesel in the top 50 based on IMDb rating:
Title: Fast Five, IMDb Rating: 7.0/10, Stars: Vin Diesel, Paul Walker, Dwayne Johnson
Title: Fast Five, IMDb Rating: 7.0/10, Stars: Vin Diesel, Paul Walker, Dwayne Johnson


Once you're done analyzing the database, prepare your presentation.

Good luck!